# Generate Lookup Table for Two Layers

In [3]:
import empymod
import numpy as np
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed
from scipy.constants import mu_0

import sys
sys.path.insert(1, '../../src')

from EM1D import EMforward2lay

## Define the geometry of the EM instrument

In [4]:
# Define Dualem-482 geometry

offsets = np.array([2, 4, 8]) # in meters
height = 0.10 # meters height From ground surface to center of coils

# Frequency
freq = 9000

## Define the ranges of conductivities and thicknesses to sample

In [5]:
# sampling of conductivities
nsl = 51 # number of samples

s0 = -3 # minimum conductivity in S/m
s1 = 0 # maximum conductivity in S/m
# conductivities array
conds = np.logspace(s0, s1, nsl)

# Sampling of 1st layer thickness
th0 = 0.1 # minimum thickness in m
th1 = 7   # maximum thickness in m
# thickness array
thicks = np.linspace(th0, th1, nsl)

## Generate the Lookup Table

In [10]:
startTime = time.time()

DataEM = Parallel(n_jobs=-1,verbose=1)(delayed(EMforward2lay)(sigma1, sigma2, thick1, offsets, height, freq) \
                                       for sigma1 in conds for sigma2 in conds for thick1 in thicks)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 5800 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 11400 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 18600 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 27400 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 37800 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 49800 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 63400 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 78600 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 95400 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 113800 tasks      | elapsed:  5.1min


Execution time in seconds: 353.959903717041


[Parallel(n_jobs=-1)]: Done 132651 out of 132651 | elapsed:  5.9min finished


## Save the Lookup Table and the sampling parameters

In [11]:
np.save('../data/LookupTable2Layers', DataEM)
np.save('../data/conds_samples', conds)
np.save('../data/thick_samples', thicks)

## Save the models sampled

In [12]:
models = []

for c1 in conds:
    for c2 in conds:
        for t1 in thicks:
            models.append([c1,c2,t1])
            
models = np.array(models)

np.save('../data/models', models)